**Retrieval-Augmented Generation (RAG)** is a method that integrates information retrieval to give generative language models additional information. 

**A typical RAG pipeline comprises of 2 main components:**

1. a **Retriever Module** that first selects relevant documents or pieces of information from a large corpus based on the input query,
2. an **Answer Generation Module** that produces more accurate and contextually relevant responses.

In [9]:
pip install langchain==0.2.5 langchain-community==0.2.5 langchain-core==0.2.7 langchain-openai==0.1.8 langchain-text-splitters==0.2.1 openai==1.34.0 bs4==0.0.2 chromadb==0.5.0 python-dotenv==1.0.0 colorama==0.4.6 tqdm==4.66.4 tiktoken==0.7.0

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [45]:

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from colorama import Fore
import warnings
warnings.filterwarnings("ignore")

client = OpenAI()
load_dotenv()
model = ChatOpenAI()

In [46]:
from langchain.prompts import PromptTemplate

# without context
prompt: str = "You are a customer support specialist who answers questions {question} and assist users with general inquiries"
prompt_template = PromptTemplate.from_template(prompt)

2. Create and run the chain

In [47]:
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

def generate_response(query):
    """Generate a response based on the query."""
    chain = (
        {"question": RunnablePassthrough()} 
        | prompt_template 
        | model 
        | StrOutputParser()
    )
    return chain.invoke(query)



Try it Out!

In [48]:
# Without RAG

response = generate_response("What did the president say about Ketanji Brown Jackson")
print(f"{Fore.GREEN}{response}{Fore.RESET}")

I'm sorry, I cannot provide real-time updates on statements made by the president. However, I can assist you with general inquiries or provide information on previous statements made by the president regarding Ketanji Brown Jackson. How can I help you today?



#### RAG Business use cases

1. Personalized Customer Support
2. Tailored Learning and Educational Materials

#### Steps to implement a RAG pipeline 
1. Indexing Documents

2. Create a vector store and store embeddings

3. Define a prompt

4. Create and run the chain

1. Load and Split Documents

In [49]:
from langchain_community.document_loaders import TextLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter 

def load_documents(datasource):
    """Load a file from path, split it into chunks, embed each chunk and load it into the vector store."""
    loader = TextLoader(datasource)
    raw_text = loader.load()
    text_splitter = RecursiveCharacterTextSplitter (chunk_size=100, chunk_overlap=0, separators=["\n\n"])
    return text_splitter.split_documents(raw_text)

2. Create and store embeddings

In [62]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from openai import OpenAI
import os

def get_embedding(text_to_embed):
    response = client.embeddings.create(
        model= "text-embedding-ada-002",
        input=[text_to_embed]
    )
    print(response.data[0].embedding)

def load_embeddings(user_query, documents, print_embeddings=False):
    """Create a vector store from a set of documents."""
    embeddings = OpenAIEmbeddings()
    db = Chroma.from_documents(documents, embeddings)
    get_embedding(user_query)
    if print_embeddings:
        _ = [get_embedding(doc.page_content) for doc in documents]
    return db.as_retriever()


In [63]:
# Load embeddings
documents = load_documents("./docs/state_of_the_union.txt")
load_embeddings("What did the president say about Ketanji Brown Jackson", documents, True)

[-0.05255802720785141, -0.008861212991178036, -0.008719054982066154, 0.0077645620331168175, 0.0037096599116921425, 0.004129366017878056, -0.0020985302980989218, -0.006654371507465839, 0.0025419292505830526, -0.004620151128619909, 0.043947283178567886, -0.004376451019197702, -0.0027094732504338026, -0.024722039699554443, -0.01908985525369644, -0.00536479102447629, 0.03514699637889862, -0.006854070350527763, 0.022379809990525246, -0.03476790711283684, 0.0052632493898272514, -0.004758925177156925, 0.015786362811923027, 0.023544155061244965, -0.0013445824151858687, 0.0005474794306792319, 0.01079727616161108, -0.026265475898981094, -0.0030225603841245174, -0.006529136560857296, -0.022014258429408073, -0.01357275154441595, -0.007331316825002432, -0.006705142557621002, -0.007554708980023861, -0.010039097629487514, -0.017790120095014572, -0.00539863808080554, 0.013863838277757168, 0.0053918687626719475, 0.022054875269532204, -0.005300481338053942, 0.009741242043673992, 0.006562984082847834, -0

KeyboardInterrupt: 

3. Define Prompt

In [70]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.prompts import ChatPromptTemplate, PromptTemplate

# with context
template: str = """/
    You are a customer support specialist /
    question: {question}. You assist users with general inquiries based on {context} /
    and  technical issues. /
    """
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template(
    input_variables=["question", "context"],
    template="{question}",
)
chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

4. Create and run the chain

In [71]:
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

def generate_response(query):
    """Generate a response using the retriever and the query."""
    documents = load_documents("./docs/state_of_the_union.txt")
    retriever = load_embeddings(query, documents)
    chain = (
        {"context": retriever, "question": RunnablePassthrough()} 
        | chat_prompt_template 
        | model 
        | StrOutputParser()
    )
    return chain.invoke(query)

Try it Out!

In [72]:
# With RAG

response = generate_response("What did the president say about Ketanji Brown Jackson")
print(f"{Fore.GREEN}{response}{Fore.RESET}")

[-0.05255802720785141, -0.008861212991178036, -0.008719054982066154, 0.0077645620331168175, 0.0037096599116921425, 0.004129366017878056, -0.0020985302980989218, -0.006654371507465839, 0.0025419292505830526, -0.004620151128619909, 0.043947283178567886, -0.004376451019197702, -0.0027094732504338026, -0.024722039699554443, -0.01908985525369644, -0.00536479102447629, 0.03514699637889862, -0.006854070350527763, 0.022379809990525246, -0.03476790711283684, 0.0052632493898272514, -0.004758925177156925, 0.015786362811923027, 0.023544155061244965, -0.0013445824151858687, 0.0005474794306792319, 0.01079727616161108, -0.026265475898981094, -0.0030225603841245174, -0.006529136560857296, -0.022014258429408073, -0.01357275154441595, -0.007331316825002432, -0.006705142557621002, -0.007554708980023861, -0.010039097629487514, -0.017790120095014572, -0.00539863808080554, 0.013863838277757168, 0.0053918687626719475, 0.022054875269532204, -0.005300481338053942, 0.009741242043673992, 0.006562984082847834, -0

**Personalized Customer Support**

> Chatbots and virtual assistants driven by RAG technology can leverage relevant customer data to provide personalized client experience and enhance customer satisfaction.

In [73]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.prompts import ChatPromptTemplate, PromptTemplate

template: str = """/
    You are a customer support specialist /
    question: {question}. You assist users with general inquiries based on {context} /
    and  technical issues. /
    By accessing specific information, I can tailor my responses. / 
    Whether you have a specific question or need help with a problem /
    I am here to assist you and enhance your experience.
    """
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template(
    input_variables=["question", "context"],
    template="{question}",
)
chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

In [74]:
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

def generate_response(query):
    """Generate a response using the retriever and the query."""
    documents = load_documents(datasource="./docs/user-manual.txt")
    retriever = load_embeddings(query, documents)
    chain = (
        {"context": retriever, "question": RunnablePassthrough()} 
        | chat_prompt_template 
        | model 
        | StrOutputParser()
    )
    return chain.invoke(query)

Try it Out!

In [75]:
response = generate_response("I have an error code E2")
print(f"{Fore.GREEN}{response}{Fore.RESET}")

[-0.015627305954694748, -0.0038546891883015633, -0.015446563251316547, -0.024010995402932167, -0.028390534222126007, 0.014181363396346569, -0.023232409730553627, -0.010239779017865658, -0.003701752983033657, -0.00817513931542635, 0.007980492897331715, 0.0031612622551620007, 0.0014546324964612722, -0.005293680354952812, -0.0187972579151392, 0.00027133154799230397, 0.006451129913330078, 0.01373645756393671, 0.012652000412344933, -0.005749013740569353, -0.004056286998093128, 0.0016136515187099576, -0.013812926597893238, -0.006944697350263596, 0.006013176403939724, -0.014487235806882381, 0.003216875484213233, -0.017420832067728043, -0.016405891627073288, -0.014807011932134628, 0.012777130119502544, -0.026916783303022385, -0.022968247532844543, -0.043656352907419205, -0.028181985020637512, -0.004876581486314535, -0.019979039207100868, 0.001525886938907206, 0.022008920088410378, -0.01136594545096159, 0.04852250963449478, 0.013131664134562016, -0.0107681043446064, -0.010705539025366306, -0.02

**Tailored Learning and Educational Materials**

> Tailored Learning and Educational Materials provide a customized approach to education, ensuring that each learner receives content that is specifically designed to meet their personal learning objectives and preferences.

In [77]:
user_profiles = [
    {
        "name": "Alice Johnson",
        "topic_preferences": ["Python", "Data Science"]
    },
    {
        "name": "Bob Smith",
        "topic_preferences": ["JavaScript", "Web Development"]
    },
    {
        "name": "Carol Davis",
        "topic_preferences": ["Java", "Python"]
    },
    {
        "name": "David Martinez",
        "topic_preferences": ["Web Development", "JavaScript"]
    },
    {
        "name": "Emily Brown",
        "topic_preferences": ["Data Science", "Python"]
    },
    {
        "name": "Frank Wilson",
        "topic_preferences": ["Java", "Data Science"]
    },
    {
        "name": "Grace Lee",
        "topic_preferences": ["Python", "Web Development"]
    },
    {
        "name": "Henry Taylor",
        "topic_preferences": ["JavaScript", "Data Science"]
    },
    {
        "name": "Irene White",
        "topic_preferences": ["Java", "Web Development"]
    },
    {
        "name": "Jack Thompson",
        "topic_preferences": ["Python", "JavaScript"]
    }
]


In [152]:
template: str = """/
    You are a chatbot expert in instructional design.
    You specialize in crafting programs and curricula based on user {profile} and specific learning needs.
    By leveraging relevant customer {data}, create personalized educational experiences.
    and return a list of recommended courses and resources. Address the user by their profile name 
    """
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template(
    input_variables=["profile", "data"],
    template="{profile}",
)
chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

In [174]:
def extract_profile(retriever, query):
    print(f"name: {query}")
    for user in user_profiles:
        if user["name"] == query:
            return retriever, user["topic_preferences"]
    return []

def give_context(retriever, user_preferences):
    return  {"data": retriever, "profile": user_preferences} 

In [175]:
from operator import itemgetter

def generate_response(query):
    """Generate a response using the retriever and the query."""
    documents = load_documents(datasource="./docs/programming-courses-and-books.txt")
    retriever = load_embeddings(query, documents)


    chain = extract_profile(retriever, query) | give_context | chat_prompt_template | model | StrOutputParser()
    
    return chain.invoke(query)

Try it Out!

In [176]:
response = generate_response("Henry Taylor")
print(f"{Fore.GREEN}{response}{Fore.RESET}")

[-0.022225650027394295, -0.012757925316691399, 0.015094640664756298, -0.025529280304908752, 0.0071175796911120415, 0.019727781414985657, -0.008453804068267345, -0.005730994511395693, 0.016665879637002945, -0.015806399285793304, -0.010333919897675514, 0.013174236752092838, -0.00918570626527071, -0.008373227901756763, 0.011381412856280804, -0.004136253613978624, 0.02054697461426258, 0.004357839003205299, 0.01725677400827408, -0.004139611031860113, -0.02814801223576069, 0.004099322948604822, 0.007305591367185116, -0.000610198185313493, -0.0015217183390632272, 0.011099395342171192, 0.033116888254880905, -0.026402192190289497, 0.019794929772615433, -0.03174709156155586, 0.011092680506408215, -0.005969366058707237, -0.01604812778532505, -0.018451988697052002, -0.024038618430495262, 5.927819074713625e-05, -0.014208300039172173, 0.01839827187359333, 0.03558789938688278, 0.01831769570708275, -0.007594323251396418, -0.006328602787107229, 0.020103804767131805, -0.0035621472634375095, -0.027610836

TypeError: unsupported operand type(s) for |: 'tuple' and 'function'